In [6]:
%cd OFA

/home/gokul.kumar/Desktop/mma/preprocessing/OFA


In [8]:
!ls

LICENSE			   colab.md	examples	  run_scripts  utils
README.md		   criterions	fairseq		  spaces.md
README_EncouragingLoss.md  data		models		  tasks
checkpoints		   datasets.md	ofa_module	  train.py
checkpoints.md		   evaluate.py	requirements.txt  trainer.py


# 1. Preparation

In [25]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fairseq import utils,tasks
from fairseq import checkpoint_utils
from utils.eval_utils import eval_step
from tasks.mm_tasks.caption import CaptionTask
from models.ofa import OFAModel
from PIL import Image

from tqdm.auto import tqdm

# Register caption task
tasks.register_task('caption',CaptionTask)

# turn on cuda if GPU is available
use_cuda = torch.cuda.is_available()
# use fp16 only when GPU is available
use_fp16 = False

# 2. Build Model

In [9]:
overrides={"bpe_dir":"utils/BPE", "eval_cider":False, "beam":5, "max_len_b":16, "no_repeat_ngram_size":3, "seed":7}
models, cfg, task = checkpoint_utils.load_model_ensemble_and_task(
        utils.split_paths('checkpoints/caption.pt'),
        arg_overrides=overrides
    )

# Move models to GPU
for model in models:
    model.eval()
    if use_fp16:
        model.half()
    if use_cuda and not cfg.distributed_training.pipeline_model_parallel:
        model.cuda()
    model.prepare_for_inference_(cfg)

# Initialize generator
generator = task.build_generator(models, cfg.generation)

2022-04-15 23:58:08 | INFO | tasks.ofa_task | source dictionary: 59457 types
2022-04-15 23:58:08 | INFO | tasks.ofa_task | target dictionary: 59457 types
/home/gokul.kumar/.conda/envs/memes/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# 3. Preprocess

In [10]:
# Image transform
from torchvision import transforms
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

patch_resize_transform = transforms.Compose([
    lambda image: image.convert("RGB"),
    transforms.Resize((cfg.task.patch_image_size, cfg.task.patch_image_size), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

# Text preprocess
bos_item = torch.LongTensor([task.src_dict.bos()])
eos_item = torch.LongTensor([task.src_dict.eos()])
pad_idx = task.src_dict.pad()
def encode_text(text, length=None, append_bos=False, append_eos=False):
    s = task.tgt_dict.encode_line(
        line=task.bpe.encode(text),
        add_if_not_exist=False,
        append_eos=False
    ).long()
    if length is not None:
        s = s[:length]
    if append_bos:
        s = torch.cat([bos_item, s])
    if append_eos:
        s = torch.cat([s, eos_item])
    return s

# Construct input for caption task
def construct_sample(image: Image):
    patch_image = patch_resize_transform(image).unsqueeze(0)
    patch_mask = torch.tensor([True])
    src_text = encode_text(" what does the image describe?", append_bos=True, append_eos=True).unsqueeze(0)
    src_length = torch.LongTensor([s.ne(pad_idx).long().sum() for s in src_text])
    sample = {
        "id":np.array(['42']),
        "net_input": {
            "src_tokens": src_text,
            "src_lengths": src_length,
            "patch_images": patch_image,
            "patch_masks": patch_mask
        }
    }
    return sample
  
# Function to turn FP32 to FP16
def apply_half(t):
    if t.dtype is torch.float32:
        return t.to(dtype=torch.half)
    return t

/tmp/ipykernel_3993005/3796615175.py:8: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  transforms.Resize((cfg.task.patch_image_size, cfg.task.patch_image_size), interpolation=Image.BICUBIC),
/home/gokul.kumar/.conda/envs/memes/lib/python3.9/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


# 4. Run Inference

In [26]:
info_fp = "../../data/hateful_memes/info_fine_grained.csv"
info_df = pd.read_csv(info_fp)
info_df.head()

,id,img,label,text,split,text_idx,pseudo_text_idx,pseudo_img_idx,gold_pc,gold_attack,...,religion_pc,sex_pc,attack_empty_attack,contempt_attack,dehumanizing_attack,exclusion_attack,inciting_violence_attack,inferiority_attack,mocking_attack,slurs_attack
0,42953,img/42953.png,0,its their character not their color that matters,train,0,4901,0,['pc_empty'],['attack_empty'],...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,train,1,3620,3486,['pc_empty'],['attack_empty'],...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13894,img/13894.png,0,putting bows on your pet,train,2,0,3956,['pc_empty'],['attack_empty'],...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,train,3,4451,4313,['pc_empty'],['attack_empty'],...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",train,4,5609,1,['pc_empty'],['attack_empty'],...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
img_folder = "../../data/hateful_memes_masked"

captions = []
for img_fn in tqdm(info_df['img'].str.split('/').str[1]):
    img_fp = os.path.join(img_folder, img_fn)

    img = Image.open(img_fp)

    # Construct input sample & preprocess for GPU if cuda available
    sample = construct_sample(img)
    sample = utils.move_to_cuda(sample) if use_cuda else sample
    sample = utils.apply_to_sample(apply_half, sample) if use_fp16 else sample

    # Run eval step for caption
    with torch.no_grad():
        result, scores = eval_step(task, generator, models, sample)

    captions.append(result[0]['caption'])

    # plt.imshow(img)
    # plt.title(result[0]['caption'])
    # plt.show()

  0%|          | 0/10000 [00:00<?, ?it/s]/home/gokul.kumar/Desktop/mma/preprocessing/OFA/models/sequence_generator.py:698: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = bbsz_idx // beam_size
100%|██████████| 10000/10000 [2:58:47<00:00,  1.07s/it] 


In [30]:
info_df['caption'] = captions
float_cols = info_df.select_dtypes(float).columns
info_df[float_cols] = info_df.select_dtypes(float).astype('Int64')

In [31]:
info_df.to_csv("../../data/hateful_memes/hateful_memes_expanded.csv")